## Data cleaning

Cleaning and preprocessing the merged dataset so that it is ready for deeper analysis.

In [7]:
import pandas as pd
import numpy as np
import glob
from src.utils import split_csv

Loading the dataset and handling missing/zero values:

In [14]:
files = sorted(glob.glob("../data/processed/merged/merged_split_part*.csv"))
df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)
print("Merged splits shape:", df.shape)

df['budget'] = df['budget'].replace(0, np.nan)
df['revenue'] = df['revenue'].replace(0, np.nan)

# Dropping rows missing important values
df = df.dropna(subset=['budget', 'revenue', 'release_date'])
df.columns

print("New shape:", df.shape)

Merged splits shape: (4803, 22)
New shape: (3229, 22)


Converting the dates:

In [15]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Extracting year and month
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

Describing profit:

In [16]:
df['profit'] = df['revenue'] - df['budget']
df['roi'] = df['revenue'] / df['budget']
print(df.head())

        budget                                             genres  \
0  237000000.0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000.0  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000.0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000.0  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000.0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id

### Parsing JSON fields

Some columns (genres, cast, crew) are stored as strings that represent lists or dictionaries. We use helper functions (parse_names, get_director) to extract:
- List of genres → with a main genre column
- Cast list → with cast size
- Director name

In [17]:
from src.utils import parse_names, get_director

df['genres'] = df['genres'].apply(parse_names)
df['main_genre'] = df['genres'].apply(lambda x: x[0] if len(x) > 0 else None) # main genre

df['cast'] = df['cast'].apply(parse_names) 
df['cast_size'] = df['cast'].apply(len) # names and size of the cast

df['director_name'] = df['crew'].apply(get_director) # director name

In [18]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,vote_count,cast,crew,release_year,release_month,profit,roi,main_genre,cast_size,director_name
0,237000000.0,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,11800,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",2009,12,2.550965e+09,11.763566,Action,83,James Cameron
1,300000000.0,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,4500,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",2007,5,6.610000e+08,3.203333,Adventure,34,Gore Verbinski
2,245000000.0,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,4466,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",2015,10,6.356746e+08,3.594590,Action,83,Sam Mendes
3,250000000.0,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,9106,"[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",2012,7,8.349391e+08,4.339756,Action,158,Christopher Nolan
4,260000000.0,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,2124,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",2012,3,2.413910e+07,1.092843,Action,27,Andrew Stanton


### Dropping potential duplicates and saving the data:

In [19]:
df = df.drop_duplicates(subset=['id'])
print('Size:', len(df))

Size: 3229


In [20]:
df.to_csv("../data/processed/movies_clean.csv", index=False)
split_csv("../data/processed/movies_clean.csv", "../data/processed/clean", "clean_split", chunk_size=1000)

Saved ../data/processed/clean\clean_split_part1.csv (1000 rows)
Saved ../data/processed/clean\clean_split_part2.csv (1000 rows)
Saved ../data/processed/clean\clean_split_part3.csv (1000 rows)
Saved ../data/processed/clean\clean_split_part4.csv (229 rows)
